# Robot Axis

<img src="image/foot_axis.png" height="500" />
<img src="image/stand_axis.png" height="500" />

## Step
<video width="500px" controls>
    <source src="image/step.mp4" type="video/mp4">
</video>

From the step movement we can expect a Acceleration on the X and Y Axis, and a Rotation around the Z Axis

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import scipy.signal
from data_analyzer import DataAnalyzer

pio.templates["default_theme"] = go.layout.Template(
    layout=go.Layout(
        font=dict(
            size=32,
            color="black",
            family="Computer Modern",
        ),
        title=dict(
            x=0.5,
            font=dict(size=34)
        ),
        xaxis=dict(title="Time (s)", showgrid=True),
        yaxis=dict(title="Measurement", showgrid=True),
        width=800,
        height=500,
    )
)

pio.templates.default = "simple_white" + "+default_theme"

In [ ]:
input_file = "../measurement/processed_data/output_toe_heel.csv"

with open(input_file, "r") as f:
        header = [next(f) for _ in range(2)]

df = pd.read_csv(input_file, header=2)
#df1 = pd.read_csv("../measurement/processed_data/output_toe_heel.csv", header=2)
#df2 = pd.read_csv("../measurement/processed_data/output_toe_heel.csv", header=2)
#df3 = pd.read_csv("../measurement/processed_data/output_toe_heel.csv", header=2)
#df4 = pd.read_csv("../measurement/processed_data/output_toe_heel.csv", header=2)
#df5 = pd.read_csv("../measurement/processed_data/output_toe_heel.csv", header=2)
#df6 = pd.read_csv("../measurement/processed_data/output_toe_heel.csv", header=2)

df["Time(s)"] = df["Time(s)"] - df.iloc[0]["Time(s)"]
df = df[df["Time(s)"] > 25]
#df["Time(s)"] = df["Time(s)"] - df.iloc[0]["Time(s)"]
#df = df[df["Time(s)"] < 15]

#df = pd.read_csv("../measurement/data/moving_all_around/output_moving_all_around.csv", header=2)
#df = pd.read_csv("../measurement/data/kanguru.csv", header=2)

df.reset_index(drop=True, inplace=True)
sampling_frequency = 1 / (df["Time(s)"].diff().mean())
print("Sampling Frequency: ", sampling_frequency)

df['Heel Button Measured'] = df['Heel Button']

df["Acceleration X (g) unfiltered"] = df["Acceleration X (g)"]
df["Acceleration Y (g) unfiltered"] = df["Acceleration Y (g)"]
df["Acceleration Z (g) unfiltered"] = df["Acceleration Z (g)"]
df["Angular Momentum X (dps) unfiltered"] = df["Angular Momentum X (dps)"]
df["Angular Momentum Y (dps) unfiltered"] = df["Angular Momentum Y (dps)"]
df["Angular Momentum Z (dps) unfiltered"] = df["Angular Momentum Z (dps)"]

cutoff_frequency = 125

num, den = scipy.signal.iirfilter(4, Wn=cutoff_frequency, fs=sampling_frequency, btype="low", ftype="butter")

df["Acceleration X (g)"] = scipy.signal.lfilter(num, den, df["Acceleration X (g) unfiltered"])
df["Acceleration Y (g)"] = scipy.signal.lfilter(num, den, df["Acceleration Y (g) unfiltered"])
df["Acceleration Z (g)"] = scipy.signal.lfilter(num, den, df["Acceleration Z (g) unfiltered"])
df["Angular Momentum X (dps)"] = scipy.signal.lfilter(num, den, df["Angular Momentum X (dps) unfiltered"])
df["Angular Momentum Y (dps)"] = scipy.signal.lfilter(num, den, df["Angular Momentum Y (dps) unfiltered"])
df["Angular Momentum Z (dps)"] = scipy.signal.lfilter(num, den, df["Angular Momentum Z (dps) unfiltered"])

df["Low Variance Signal"] = 6 * (df["Angular Momentum Z (dps)"] / 400) * (df["Acceleration Y (g)"] / 16)

In [ ]:
figure = go.Figure()

df = DataAnalyzer.fix_button_detection(df)

figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Heel Button Measured"], mode="lines", name="Heel Button Measured"))
figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Heel Button"], mode="lines", name="Heel Button"))
#figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Acceleration X (g)"] / 16, mode="lines", name="Acceleration X (g)"))
figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Low Variance Signal"], mode="lines", name="Mixed"))

figure.update_layout(title=dict(text="Button"))

figure.show()

In [ ]:
df = DataAnalyzer.analytical_step_detection(df, threshold=0.2, threshold2=0.2, threshold4=0.1)
#df = DataAnalyzer.analytical_step_detection(df, threshold=0.02, threshold2=0.2)

figure = go.Figure()

figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Stand detected"], mode="lines", name="Stand detected"))
#figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Flight detected"], mode="lines", name="Flight detected"))
#figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Stand init detected"], mode="lines", name="Stand init detected"))

#figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Acceleration X (g)"] / 16, mode="lines", name="Acceleration X (g)"))
#figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Acceleration Y (g)"] / 16, mode="lines", name="Acceleration Y (g)"))
#figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Angular Momentum Z (dps)"] / 400, mode="lines", name="Angular Momentum Z (dps)"))
figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Low Variance Signal"], mode="lines", name="Mixed"))

#figure.add_trace(go.Scatter(x=df["Time(s)"], y=df["Heel Button"], mode="lines", name="Heel Button"))
figure.add_trace(go.Scatter(x=df["Time(s)"], y=abs(df["Heel Button"] - df["Stand detected"]), mode="lines", name="Difference"))

equal_values = (df["Heel Button"] == df["Stand detected"]).sum()

print(f"{equal_values} out of {df.shape[0]} entries are equal, which is {equal_values / df.shape[0] * 100:.2f}%")

figure.update_layout(title=dict(text="Analytical Step Detection"))

figure.show()

In [ ]:
df_save = df.copy()

df_save["Acceleration X (g)"] = df_save["Acceleration X (g) unfiltered"]
df_save["Acceleration Y (g)"] = df_save["Acceleration Y (g) unfiltered"]
df_save["Acceleration Z (g)"] = df_save["Acceleration Z (g) unfiltered"]
df_save["Angular Momentum X (dps)"] = df_save["Angular Momentum X (dps) unfiltered"]
df_save["Angular Momentum Y (dps)"] = df_save["Angular Momentum Y (dps) unfiltered"]
df_save["Angular Momentum Z (dps)"] = df_save["Angular Momentum Z (dps) unfiltered"]

# drop unfiltered columns
df_save = df_save.drop(columns=["Acceleration X (g) unfiltered", "Acceleration Y (g) unfiltered", "Acceleration Z (g) unfiltered", "Angular Momentum X (dps) unfiltered", "Angular Momentum Y (dps) unfiltered", "Angular Momentum Z (dps) unfiltered"])

out_file = "../measurement/processed_data/test.csv"

def save_with_header(df, filename):
        with open(filename, "w", newline="") as f:
            f.writelines(header)
            df.to_csv(f, index=False)

save_with_header(df_save, out_file)